In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [35]:
def adjustDataframe(dataFrame):
##  I think I was splitting data incorrectly, as Male and Female created two columns
    dataFrame['Sex'].replace({0:'Female',1:'Male'},inplace=True)
##  Get name prefix
    namePrefix = dataFrame["Name"].str.split(', ', 1).str[1]
    namePrefix = namePrefix.str.split('. ',1).str[0]
    dataFrame["NamePrefix"] = namePrefix
##  Split ticket into numeric and prefix parts
    spacedTicket = ' ' + dataFrame['Ticket'].astype(str)
    ticketSplit = spacedTicket.str.rsplit(' ',1,expand=True)
    dataFrame["TicketPrefix"] = ticketSplit[0]
    dataFrame["TicketPrefix"] = dataFrame["TicketPrefix"].str.replace(".","")
    dataFrame["TicketPrefix"] = dataFrame["TicketPrefix"].str.replace(" ","")
    dataFrame["TicketNumber"] = ticketSplit[1]  
#Check if there were NaN for those columns
    dataFrame["FamilySize"] = dataFrame["SibSp"] + dataFrame["Parch"]  + 1

## There are multiple cabins per some of passengers - how to proceed?
# For now I will only use deck name and amount of cabins. For later I should check how could I use multiple entries
    dataFrame["CabinDeck"] = dataFrame["Cabin"].str[:1]
    dataFrame["CabinsAmount"] = dataFrame["Cabin"].str.split(" ",-1).str.len()

    del namePrefix, spacedTicket, ticketSplit
    dataFrame.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
    return dataFrame

In [36]:
def extract_columns(columns):
    yColumn = "Survived"
    dfTrain = pd.read_csv("./train.csv")
    dfTest = pd.read_csv("./test.csv")

    y = dfTrain[yColumn]
    dfTrain.drop(yColumn, axis=1, inplace=True)
    X_full = dfTrain.append(dfTest, ignore_index=True)
    #   Enhance dataframe with additional columns
    X_full = adjustDataframe(X_full)
    # Fill nan values
    X_full = X_full.replace(r'^\s*$', np.nan, regex=True)
    # Impute nan to mean and encode categorical data into separate columns
    numericColumns, categoricalColumns = getValidColumns(columns)
    mean_Imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_numeric = mean_Imputer.fit_transform(X_full[numericColumns])
    X_numeric = StandardScaler().fit_transform(X_numeric)
    categorical_data = X_full[categoricalColumns].fillna("N/A").astype(str)
    encoder = DictVectorizer(sparse = False)
    X_categorical = encoder.fit_transform(categorical_data.T.to_dict().values())    

    # Split data back to train and test
    X_train = np.hstack((X_numeric[0:891,:],X_categorical[0:891,:]))
    X_final_test = np.hstack((X_numeric[891:1309,:],X_categorical[891:1309,:]))
    return X_train, X_final_test, y

In [39]:
# build column list
def getValidColumns(columns):
    allNumericColumns = ["Pclass", "Age", "FamilySize", "Fare", "CabinsAmount", "Sex"]
    allCategoricalColumns = ["NamePrefix", "TicketPrefix", "CabinDeck", "Embarked"]
    numColumns = []
    catColumns = []

    for column in columns:
        if column in allNumericColumns:
            numColumns.append(column)
        if column in allCategoricalColumns:
            catColumns.append(column)
    return numColumns, catColumns


In [38]:
used_columns= ["Pclass", "Age", "FamilySize", "Fare", "CabinsAmount","Sex"]
X_train, X_test, y = extract_columns(used_columns)
columnsAmount = X_train.shape[1]
for column in range(columnsAmount):
    print(" coef: ",round(np.corrcoef(X_train[:, column], y) [1,0],3))
    # print(used_columns[column])
    # print(round(np.corrcoef(X_train[:, column], y)[1,0],3))


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'male'